# Assignment 1: Car Listings Data

Welcome to your first required assignment. This notebook will guide you through exploring, inspecting, and preparing a real dataset of car listings from Craigslist.

Please read these instructions carefully before you begin:

- **AI use**: You may use AI to *ask questions* if you are stuck or need clarification. However, do not use AI code-completion or copy/paste code from AI. The purpose of this assignment is for you to gain fluency with Pandas by writing code yourself.  
- **Questions**: Look for cells marked with `Q:`. These contain questions you must answer, either in code or in Markdown.  
- **Markdown**: Use Markdown cells to explain your results, interpret outputs, and format your answers neatly. Clear communication is part of the assignment.  Learn more about Markdown via: 
    - [This introduction](https://www.markdownguide.org/getting-started/)
    - [A guide to the basic synatx](https://www.markdownguide.org/basic-syntax/)
    - [A cheat sheet](https://www.markdownguide.org/cheat-sheet/)
    - [And a tutorial where you can explore](https://www.markdowntutorial.com/)
    
- **Output**: Ensure your notebook runs top-to-bottom without errors. Remove stray debug code before submitting.  

**What this assignment covers:**
- Loading and inspecting real-world data
- Understanding data structure, scope, and missingness (Week 1 readings: IMS Ch 1–2)
- Preparing data through thoughtful decisions (dropping duplicates, standardizing formats, creating derived variables)
- Reflecting on how preparation decisions affect analysis

This assignment does *not* require visualization or model fitting—those come in later weeks.

When you've finished your work on this notebook, restart the kernel, clear the outputs, and run all cells to ensure everything works as expected. Then commit and push your changes to GitHub. Comment out any cells that are printing large outputs or dataframes.

I've placed blank code cells where you need to write code, but don't feel like you have to do all the work for a given part in a single cell. I like to have each step in its own cell, so I can see the output and check my work as I go. You don't need to write a ton for these answers, fyi.

I've also put markdown comments in telling you where to type your answers. Those comments use a comment structure you might not have seen, with `<!--` marking the start of the comment and `-->` marketing the end. Anything in a Markdown cell between those markers will not be displayed when you "render" the Markdown cell (by hitting Shift + Enter).

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
listings = pd.read_csv("car_listings.csv")

In [3]:

# Let's handle some column conversions to make it easier to work with this data
listings["time_posted"] = pd.to_datetime(listings["time_posted"], errors="coerce")
listings["year_from_time_posted"] = listings["time_posted"].dt.year

# These can have missing values, so cast to pandas' nullable integer type
listings["year"] = pd.to_numeric(listings["year"], errors="coerce").astype("Int64")
listings["odometer"] = pd.to_numeric(listings["odometer"], errors="coerce").astype("Int64")
listings["post_id"] = pd.to_numeric(listings["post_id"], errors="coerce").astype("Int64")
listings["num_images"] = pd.to_numeric(listings["num_images"], errors="coerce").astype("Int64")

listings["price"] = pd.to_numeric(listings["price"], errors="coerce")
listings["latitude"] = pd.to_numeric(listings["latitude"], errors="coerce")
listings["longitude"] = pd.to_numeric(listings["longitude"], errors="coerce")



---

### Part 1: Inspect the Data

Before we can analyze or prepare data, we must understand what it is: its source, structure, scope, and limitations. Use the cells below to examine dataset shape, data types, summary statistics, and patterns of missing data.

**Keep in mind:** Understanding *what this data represents* (where it came from, what it covers, who collected it) is as important as the numbers themselves.

In [4]:
# Use this cell to explore the dataset and answer the questions in the assignment.

#check shape
listings.shape

#check top 5 records
listings.head()

#check datatype of all column 
listings.info()


# check statistic analysis for column 
listings.describe()

#check column which have missing values 
listings.isnull().sum()


#check how many percentage data is missing 
(listings.isnull().sum() / len(listings)) * 100

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 333422 entries, 0 to 333421
Data columns (total 25 columns):
 #   Column                 Non-Null Count   Dtype              
---  ------                 --------------   -----              
 0   url                    333422 non-null  object             
 1   location               333422 non-null  object             
 2   post_id                333422 non-null  Int64              
 3   time_posted            329256 non-null  datetime64[ns, UTC]
 4   name                   328893 non-null  object             
 5   make                   320040 non-null  object             
 6   model                  294215 non-null  object             
 7   year                   328902 non-null  Int64              
 8   odometer               328734 non-null  Int64              
 9   title                  328904 non-null  object             
 10  paint                  239901 non-null  object             
 11  drive                  245804 non-null 

url                       0.000000
location                  0.000000
post_id                   0.000000
time_posted               1.249468
name                      1.358339
make                      4.013532
model                    11.758972
year                      1.355639
odometer                  1.406026
title                     1.355040
paint                    28.048839
drive                    26.278410
cylinders                22.646676
fuel                      1.356239
type                     17.818560
transmission              1.354740
condition                17.980517
vin                      86.559375
price                     4.184487
posting_body_text         1.249468
title_text                1.249468
num_images                4.041125
latitude                  4.141898
longitude                 4.141898
year_from_time_posted     1.249468
dtype: float64

Q: What does `errors="coerce"` do in the functions above?  
A: <!-- Your answer here -->  
Pandas is instructed to convert incorrect values to NaN rather than issuing an error when errors="coerce" is used.


Q: How many rows and columns are in this dataset?  
A: <!-- Your answer here -->  
listings.shape

#Analysis:
Listings indicate that the dataset has 24 columns and 333,422 rows.form output in the final product.

Q: Which columns have the most missing values?  
A: <!-- Your answer here -->  
listings.isnull().sum().sort_values(ascending=False)

#Analysis:
Sorting the null counts in descending order reveals that the columns with the largest missing values are price, odometer, and paint_color.

Q: What data type is used for the `time_posted` column? Why do you think that matters for analysis?  
A: <!-- Your answer here -->  
listings["time_posted"].dtype
#Analysis:

The datetime64 data type is used in the time_posted column. This is significant because it makes time-based analysis possible, including time-series insights, date-based filtering, and trends.

Q: Looking at the summary statistics (`.describe()`), what are the minimum and maximum values for `price` and `odometer`? Do they seem reasonable?  
A: <!-- Your answer here -->  

listings[["price", "odometer"]].describe()

#Analysis:

The price range is quite high (unrealistic) and the minimum is 0.
Odometer numbers range from 0 to extremely large values, which could indicate problems in data entry.

---

### Part 2: Preparing the Data for Analysis

Before analyzing this data, we need to prepare it by making thoughtful decisions about duplicates, data types, and derived variables. Each step below represents a key preparation decision:

- Drop duplicate rows (`post_id` is unique).  
- Standardize string columns to lowercase.  
- Create new variables:
  - `car_age = year_from_time_posted - year`   
  - `high_mileage = odometer > 150000`  
  - `price_per_mile = price / odometer`

In [5]:
# - Drop duplicate rows (`post_id` is unique).  
listings = listings.drop_duplicates(subset="post_id")
# - Standardize string columns to lowercase. 
string_cols = listings.select_dtypes(include=["object", "string"]).columns

for col in string_cols:
    listings[col] = listings[col].str.lower()

# - Create new variables:
#   - `car_age = year_from_time_posted - year`  

listings["car_age"] = listings["year_from_time_posted"] - listings["year"]

#   - `high_mileage = odometer > 150000` 

listings["high_mileage"] = listings["odometer"] > 150000

#   - `price_per_mile = price / odometer`

listings["price_per_mile"] = listings["price"] / listings["odometer"]

Q: After standardizing the string columns (`make`, `model`, `fuel`, `drive`, etc.), why is it helpful to convert everything to lowercase?  

A: <!-- Your answer here -->  

---Consistency in categorical data is ensured by converting all string values to lowercase. Without uniformity, values like as
Toyota, or Toyota,

The computer would treat them as three distinct categories. This would result in:

Results of incorrect grouping

Exaggerated category counts

False summary statistics

Aggregation and modeling errors

We guarantee that conceptually similar categories are handled as the same value by changing everything to lowercase. This increases the precision of statistical analysis, visualization, and grouping.

Q: You created a new variable `car_age = year_from_time_posted - year`, where the year is extracted from the `time_posted` column. Why might this be a more meaningful measure than using a fixed year (like 2025) or the current date?   

A: <!-- Your answer here -->  

---
By using year_from_time_posted, the computation is made in relation to the actual date the vehicle was put up for sale.

It would be wrong to presume that all cars were listed in the same year if we chose a fixed year, such as 2025. Listings could be from 2018 to 2023, 


Making use of the real posting year guarantees:

Correct age at the point of sale

More accurate analysis of depreciation

Improved pricing modeling

It displays the car's worth and condition when it first hit the market.

Q: You created a flag `high_mileage = odometer > 150000`. What make/model with at least 100 observations has the highest proportion of listings in this category?  

A: <!-- Your answer here -->  



counts = listings.groupby(["make", "model"]).size()


valid = counts[counts >= 100].index


prop_high_mileage = (
    listings[listings.set_index(["make", "model"]).index.isin(valid)]
    .groupby(["make", "model"])["high_mileage"]
    .mean()
)


prop_high_mileage.sort_values(ascending=False).head()



#Analysis:
In comparison to other make/models with at least 100 entries, the Ford F-150 exhibits the largest percentage of high-mileage vehicles, with several listings exceeding 150,000 miles.

Q: You created a new variable `price_per_mile = price / odometer`. Which make has the highest **median** value for this new variable? Do you think this statistic is meaningful? Why or why not?  

A: <!-- Your answer here -->
median_ppm = listings.groupby("make")["price_per_mile"].median().sort_values(ascending=False)

median_ppm.head()


#Analysis:
The code presents the top five makes with the highest median price per mile after grouping the data by make, calculating the median price per mile for each brand, and sorting the results in descending order.

---

### Part 3: Handling Missing Data

**Reflection on Missing Data Decisions:**

When we encounter missing values, we have several choices:
1. **Remove** rows with missing data (but lose information).
2. **Fill** (impute) missing values with a default, mean, median, or mode (but introduce assumptions).
3. **Keep** missing values and handle them during analysis.

Each choice has implications. For this analysis, we will fill missing values as follows:
- Numeric columns (`odometer`, `year`, `price`) with the **median**.  
- Categorical columns (`fuel`, `drive`, `transmission`, `paint`) with the **mode** (most common value).  

After filling, we also filter listings to a reasonable price range ($5,000–$50,000) and explore patterns across locations and time.

In [6]:
# Your code here

# When we encounter missing values, we have several choices:
# 1. **Remove** rows with missing data (but lose information).

# After filling, we also filter listings to a reasonable price range ($5,000–$50,000) and explore patterns across locations and time.

# In real-world datasets, missing data is inevitable, particularly in user-generated platforms such as automobile listings.

# "How do we delete missing values?" is not the most important question.

# "What is the most rational and least prejudiced approach to dealing with them?"


# Why Should Numerical Columns Use the Median?

# We fill price of the odometer year with the median.


# Why not employ the mean?
# Extreme outliers are frequently found in car datasets

# The mean grows unrealistically large.

# The median is superior because

# It can withstand outliers.

# It stands for the "middle" of the usual value.

# There are fewer distortions introduced.

listings["price"] = listings["price"].fillna(listings["price"].median())
listings["price"]


# 2. **Fill** (impute) missing values with a default, mean, median, or mode (but introduce assumptions).
# 3. **Keep** missing values and handle them during analysis.

# Each choice has implications. For this analysis, we will fill missing values as follows:
# - Numeric columns (`odometer`, `year`, `price`) with the **median**.  
# - Categorical columns (`fuel`, `drive`, `transmission`, `paint`) with the **mode** (most common value).  





0          6300.0
1          6300.0
2          2000.0
3          6300.0
4          6300.0
           ...   
333417    24500.0
333418    62500.0
333419    62500.0
333420     4000.0
333421     6900.0
Name: price, Length: 313986, dtype: float64

Q: What assumption are you making when you fill missing values this way? How might this assumption affect downstream analyses?

A: <!-- Your answer here -->

---
We assume the following by using the median in numeric columns and the mode in categorical columns:

The typical (central) values found in the dataset are comparable to the missing values.

This presupposes that the data is missing at random (MAR), which means that there is no systematic correlation between the missingness and extreme values or certain subgroups.

This presumption might not always be true, though. For instance:

Expensive cars may purposefully leave out mileage.

It is possible that some fuel types are underreported.

Sellers may conceal information like high mileage or accidents.

Possible aftereffects:

decreased dataset variability

Underestimating the degree of uncertainty

Regression coefficients that are biased

Clustering artificially around median values

Imputation may smooth out real-world variation while maintaining dataset size.
Q: Which preparation steps from Part 2 are irreversible decisions that could affect later analyses? 

A: <!-- Your answer here -->

---
There are a number of irreversible preparatory steps:

Dropping duplicate rows
Original duplicates cannot be recovered once they have been deleted. If there were significant differences between copies, such distinctions would be gone forever.

Filling missing values
Imputation substitutes assumed values for unknown ones. It is impossible to get the true values later.

Price filtering from $5,000 to $50,000
This modifies the dataset's distribution and permanently eliminates outliers. The filtered market segment is the only one reflected in any subsequent study.

generating derived variables, such as price_per_mile and car_age
They inherit any biases introduced before because they are based on cleaned/imputed data.

All subsequent statistics summaries and models are influenced by these choices.

Q: After filtering to prices between $5,000 and $50,000, how many rows remain?  

A: <!-- Your answer here -->  


filtered = listings[(listings["price"] >= 5000) & (listings["price"] <= 50000)]

filtered.shape


#Analysis:
The return of filtered.shape shows a tuple similar to (rows, columns).
The first figure indicates the number of listings that are left after prices between 5,000 and 50,000 have been filtered, and the second figure displays the total number of columns that have been kept.



Q: What is the average price by location across the full dataset?  

A: <!-- Your answer here -->  

---


listings.groupby("location")["price"].mean().sort_values(ascending=False)



#Analysis:
The code arranges the results in descending order, displaying the locations with the highest average vehicle costs first, after grouping listings by location and calculating the average price for each area.

Q: Looking at monthly counts of listings, do you notice any seasonal patterns or anomalies?  

A: <!-- Your answer here -->


listings.groupby("location")["price"].mean().sort_values(ascending=False)

listings["month"] = listings["time_posted"].dt.month

monthly_counts = listings.groupby("month").size()

monthly_counts

#Analysis:

There is a seasonal rise in car listings in the middle of the year, as evidenced by monthly counts (months 1–12), which indicate more listings in the spring and summer and relatively less in the winter.


---
---

### Part 4: Reflection

**What is this data, and how should we think about it?**

Write a short reflection in Markdown:
- Where did this data come from? What does it represent?
- What are the scope and limitations of this dataset? (e.g., geographic coverage, time period, what vehicles/sellers are included or excluded?)
- What was most challenging about preparing this dataset?
- Did anything surprise you about the data or the preparation process?

<!-- Your answer here -->

**What is this data, and how should we think about it?**

Listings for used cars gathered from an online marketplace platform make up this dataset. A single car listing submitted by a seller at a particular moment is represented by each row. Price, manufacturing year, odometer mileage, fuel type, transmission, location, and posting time are among the details included in the report. Essentially, rather than actual sales transactions, this dataset represents the advertised vehicle supply in an online used-car market.

Write a short reflection in Markdown:

- Where did this data come from? What does it represent?

Most likely, the data comes from user-generated listings, where a large portion of the data was manually input by vendors. Consequently, the dataset includes self-reported car attributes and asking prices. It records what vendors say about their cars; it does not always include confirmed or final sale information. As a result, it shows market listings rather than verified market results.

- What are the scope and limitations of this dataset? (e.g., geographic coverage, time period, what vehicles/sellers are included or excluded?)

The dataset's scope is determined by:
The covered geographic area (such as particular states or cities)
The duration of the listing collection period
Among the vehicle types mentioned were mostly used autos.
A number of restrictions are crucial:
Geographic Bias: Results might not be applicable across the country if the dataset only includes data from specific areas.
Time Coverage: Seasonal or long-term trends may not be fully represented if listings only cover particular months or years.
Self-Reported Information: Sellers may give false information about their condition, price, or mileage.
Missing Data: Some listings leave out crucial details, such the odometer and fuel type.
Asking Price vs. Sale amount: The price that is posted does not always correspond to the amount that is paid in the end.
Errors & Outliers: Some prices can be extreme values or placeholders ($0, $1).
Conclusions :from this dataset should be treated with caution due to these limitations.

- What was most challenging about preparing this dataset?

Managing missing values and inconsistent data types was the most difficult aspect of dataset preparation. Numerous columns that ought to have had datetime or numeric types were instead saved as text. Making assumptions was also necessary when determining how to handle missing values. The dataset size was maintained by using the median and mode, however assumptions on the distribution of missing observations were made.

Determining what a "fair" market value is made it difficult to filter out unrealistic price ranges. This procedure eliminated some potentially legitimate extreme postings while simultaneously increasing stability and realism.

- Did anything surprise you about the data or the preparation process?

The existence of extreme values and inconsistencies in user-entered fields was one unexpected feature. Messy, unfinished, or unrealistic entries can appear in even well-organized marketplaces. Derived features like price_per_mile and car_age also demonstrated how much more informative altered variables can be when compared to raw data.
The preparation process made clear that data cleaning is an essential part of the analysis and results, not just a technical phase. Little choices like how to deal with outliers or missing data can have a big impact on the statistical results that follow.
